In [1]:
import pandas as pd
from fugue_notebook import setup
import seaborn as sns
setup()

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [14]:
%%fsql
SELECT * FROM df WHERE sex='male' ORDER BY charges DESC LIMIT 5
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
1146,60,male,32.800,0,yes,southwest,52590.82939
34,28,male,36.400,1,yes,southwest,51194.55914
1241,64,male,36.960,2,yes,southeast,49577.66240


In [16]:
%%fsql
SELECT 
    sex, AVG(bmi) AS avg_bmi 
FROM 
    df 
GROUP BY 
    sex
PRINT

,sex:str,avg_bmi:double
0,female,30.377749
1,male,30.943129


Manual Create

In [17]:
%%fsql
CREATE [[1, 'John', 54], [2, 'Jane', 45]] SCHEMA id:int, name:str, score:float
PRINT

,id:int,name:str,score:float
0,1,John,54.0
1,2,Jane,45.0


From List

In [18]:
data = [
    [1, 'John', 45.5],
    [2, 'JohnDoe', 45.6],
    [3, 'JohnCa', 45.7],
    [4, 'JohnWay', 45.8],
    [5, 'JohnCane', 45.9],
    [6, 'JohnWayne', 46],
    [7, 'JohnDow', 46.1],
    [8, 'JohnCayne', 46.2],
    [9, 'JohnWy', 46.3],
    [10, 'JohnRu', 46.4],
    [11, 'JohnWitch', 46.5]
]

In [19]:
%%fsql
CREATE {{data}} SCHEMA id:int, name:str, score:float
PRINT

,id:int,name:str,score:float
0,1,John,45.500000
1,2,JohnDoe,45.599998
2,3,JohnCa,45.700001
3,4,JohnWay,45.799999
4,5,JohnCane,45.900002
5,6,JohnWayne,46.000000
6,7,JohnDow,46.099998
7,8,JohnCayne,46.200001
8,9,JohnWy,46.299999
9,10,JohnRu,46.400002


From DataFrame

In [22]:
df_iris = sns.load_dataset('iris')
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [27]:
data_iris = df_iris[['petal_width', 'petal_length']].values.tolist()

In [29]:
%%fsql
CREATE {{data_iris}} SCHEMA petal_width:float, petal_length:float

READ DATASET BY THE FUGUESQL

In [4]:
%%fsql
df = LOAD 'data/insurance.csv' (header=TRUE, infer_schema=TRUE)
SELECT * FROM df 
LIMIT 5
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


QUERIES

In [5]:
iris = sns.load_dataset('iris')

In [6]:
%%fsql
SELECT
    petal_length,
    petal_width
FROM
    iris
WHERE
    species='setosa'
LIMIT 5
PRINT

,petal_length:double,petal_width:double
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2


In [7]:
%%fsql
SELECT
    species,
    AVG(petal_length) AS avg_plength,
    AVG(petal_width) AS avg_pwidth
FROM
    iris
GROUP BY
    species
PRINT

,species:str,avg_plength:double,avg_pwidth:double
0,setosa,1.462,0.246
1,versicolor,4.260,1.326
2,virginica,5.552,2.026


In [12]:
df_tips = sns.load_dataset('tips')
df_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [14]:
#Convert category to string type for required columns
changed_columns = ['sex', 'smoker', 'day', 'time']
for col in changed_columns:
    df_tips[col] = df_tips[col].astype('str')
df_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [17]:
%%fsql
SELECT 
    * 
FROM 
    df_tips 
WHERE 
    sex='Female' AND 
    smoker='Yes'
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long
67,3.07,1.00,Female,Yes,Sat,Dinner,1
72,26.86,3.14,Female,Yes,Sat,Dinner,2
73,25.28,5.00,Female,Yes,Sat,Dinner,2
92,5.75,1.00,Female,Yes,Fri,Dinner,2
93,16.32,4.30,Female,Yes,Fri,Dinner,2
100,11.35,2.50,Female,Yes,Fri,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2
102,44.30,2.50,Female,Yes,Sat,Dinner,3
103,22.42,3.48,Female,Yes,Sat,Dinner,2
109,14.31,4.00,Female,Yes,Sat,Dinner,2


In [20]:
%%fsql
SELECT 
    sex,
    smoker,
    AVG(tip) AS avg_tip,
    COUNT(*) AS count
FROM 
    df_tips 
GROUP BY
    sex, smoker
PRINT

,sex:str,smoker:str,avg_tip:double,count:long
0,Female,No,2.773519,54
1,Female,Yes,2.931515,33
2,Male,No,3.113402,97
3,Male,Yes,3.051167,60


In [30]:
%%fsql
SELECT 
    * 
FROM 
    df_tips 
WHERE 
    sex='Female' AND 
    smoker='Yes'
ORDER BY
    total_bill DESC
LIMIT
    5
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long
102,44.30,2.50,Female,Yes,Sat,Dinner,3
197,43.11,5.00,Female,Yes,Thur,Lunch,4
219,30.14,3.09,Female,Yes,Sat,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2


In [31]:
%%fsql
SELECT
    MAX(total_bill)
FROM
    df_tips
PRINT

,_942cd7df35fa:double
0,50.81


In [32]:
%%fsql
SELECT
    day,
    SUM(tip) AS sum_tip,
    SUM(total_bill) AS sum_total_bill
FROM
    df_tips
GROUP BY
    day
PRINT

,day:str,sum_tip:double,sum_total_bill:double
0,Fri,51.96,325.88
1,Sat,260.40,1778.40
2,Sun,247.39,1627.16
3,Thur,171.83,1096.33


In [33]:
%%fsql
SELECT
    day,
    SUM(tip) AS sum_tip,
    SUM(total_bill) AS sum_total_bill
FROM
    df_tips
GROUP BY
    day
HAVING
    SUM(total_bill) <= 1000
PRINT

,day:str,sum_tip:double,sum_total_bill:double
0,Fri,51.96,325.88


WINDOW FUNCTION

In [35]:
%%fsql
SELECT
    total_bill,
    SUM(total_bill) OVER() AS sum_total_bill,
    tip,
    SUM(tip) OVER() AS sum_tip
FROM
    df_tips
LIMIT 9
PRINT

,total_bill:double,sum_total_bill:double,tip:double,sum_tip:double
0,16.99,4827.77,1.01,731.58
1,10.34,4827.77,1.66,731.58
2,21.01,4827.77,3.50,731.58
3,23.68,4827.77,3.31,731.58
4,24.59,4827.77,3.61,731.58
5,25.29,4827.77,4.71,731.58
6,8.77,4827.77,2.00,731.58
7,26.88,4827.77,3.12,731.58
8,15.04,4827.77,1.96,731.58


In [38]:
%%fsql
SELECT
    sex,
    total_bill,
    SUM(total_bill) OVER(PARTITION BY sex) AS sum_total_bill
FROM
    df_tips
LIMIT 9
PRINT

,sex:str,total_bill:double,sum_total_bill:double
0,Female,16.99,1570.95
1,Male,10.34,3256.82
2,Male,21.01,3256.82
3,Male,23.68,3256.82
4,Female,24.59,1570.95
5,Male,25.29,3256.82
6,Male,8.77,3256.82
7,Male,26.88,3256.82
8,Male,15.04,3256.82


VARIABLE FUGUESQL

    Python Variable

In [52]:
df = pd.read_csv('data/insurance.csv')
age = 20

In [53]:
%%fsql
SELECT 
    * 
FROM 
    df 
WHERE 
    age <= {{age}}
LIMIT 5
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
15,19,male,24.600,1,no,southwest,1837.23700
22,18,male,34.100,0,no,southeast,1137.01100
31,18,female,26.315,0,no,northeast,2198.18985


        Local Variable

In [59]:
%%fsql
df_female = SELECT * FROM df WHERE sex='female'
df_female_smoker = SELECT * FROM df_female WHERE smoker='yes'
PRINT df_female_smoker

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.90,0,yes,southwest,16884.9240
11,62,female,26.29,0,yes,southeast,27808.7251
23,34,female,31.92,1,yes,northeast,37701.8768
58,53,female,22.88,1,yes,southeast,23244.7902
64,20,female,22.42,0,yes,northwest,14711.7438
70,27,female,24.75,0,yes,southeast,16577.7795
84,37,female,34.80,2,yes,southwest,39836.5190
86,57,female,31.16,0,yes,northwest,43578.9394
94,64,female,31.30,2,yes,southwest,47291.0550
103,61,female,29.92,3,yes,southeast,30942.1918


In [43]:
%%fsql
df_female = SELECT * FROM df WHERE sex='female'
SELECT * FROM df_female WHERE age <= {{age}}
LIMIT 5
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
31,18,female,26.315,0,no,northeast,2198.18985
32,19,female,28.600,5,no,southwest,4687.79700
46,18,female,38.665,2,no,northeast,3393.35635
50,18,female,35.625,0,no,northeast,2211.13075


        Global Variable

In [60]:
iris = sns.load_dataset('iris')

In [61]:
%%fsql
SELECT * FROM iris
WHERE
    species='setosa'
YIELD DATAFRAME AS mydata

In [62]:
%%fsql
SELECT * FROM mydata 
WHERE petal_width=0.2
LIMIT 5
PRINT

,sepal_length:double,sepal_width:double,petal_length:double,petal_width:double,species:str
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [63]:
df = mydata.as_pandas()

In [64]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [67]:
%%fsql
SELECT * FROM df_tips
WHERE
    sex='Female'
YIELD DATAFRAME AS df_female

In [68]:
%%fsql
SELECT * FROM df_female
WHERE
    smoker='Yes'
YIELD DATAFRAME AS female_smoker

In [71]:
df_female_smoker_pandas = female_smoker.as_pandas()
df_female_smoker_pandas.head()

,total_bill,tip,sex,smoker,day,time,size
67,3.07,1.00,Female,Yes,Sat,Dinner,1
72,26.86,3.14,Female,Yes,Sat,Dinner,2
73,25.28,5.00,Female,Yes,Sat,Dinner,2
92,5.75,1.00,Female,Yes,Fri,Dinner,2
93,16.32,4.30,Female,Yes,Fri,Dinner,2


        Change data type of the column

In [74]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [75]:
columns = ['class', 'deck']
for col in columns:
    titanic[col] = titanic[col].astype('str')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         891 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


RENAME COLUMN

In [82]:
%%fsql
df1 = SELECT * FROM titanic
df2 = RENAME COLUMNS sex:gender FROM df1
PRINT df2

,survived:long,pclass:long,gender:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,nan,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,nan,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,nan,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,nan,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,nan,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,nan,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,nan,Cherbourg,yes,False


CHANGE DATATYPE

In [83]:
%%fsql
df1 = SELECT * FROM titanic
df2 = ALTER COLUMNS survived:int, pclass:int, age:float FROM df1
PRINT df2

,survived:int,pclass:int,sex:str,age:float,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,nan,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,nan,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,nan,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,nan,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,nan,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,nan,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,nan,Cherbourg,yes,False


DROP COLUMN

In [84]:
%%fsql
df1 = SELECT * FROM titanic
df2 = DROP COLUMNS class, who, deck, embark_town IF EXISTS FROM df1
PRINT df2

,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,adult_male:bool,alive:str,alone:bool
0,0,3,male,22.0,1,0,7.2500,S,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,True,no,True
5,0,3,male,NaN,0,0,8.4583,Q,True,no,True
6,0,1,male,54.0,0,0,51.8625,S,True,no,True
7,0,3,male,2.0,3,1,21.0750,S,False,no,False
8,1,3,female,27.0,0,2,11.1333,S,False,yes,False
9,1,2,female,14.0,1,0,30.0708,C,False,yes,False


CHANGE STRUCTURE OF THE TABLE

In [88]:
%%fsql
df1 = SELECT * FROM titanic
SELECT * FROM df1 WHERE age IS NULL
PRINT 2 ROW
df2 = DROP ROWS IF ANY NULL ON age FROM df1
SELECT * FROM df2 WHERE age IS NULL
PRINT 

,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,nan,Queenstown,no,True
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,nan,Southampton,yes,True


,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
